In [ ]:
import numpy as np
import pandas as pd
from preprocessing import genres
import IPython.display as ipd
from convolutional import ConvolutionalNN


def cosine_similarity(a, b):
    return np.dot(a.flatten(), b.flatten()) / (np.linalg.norm(a) * np.linalg.norm(b))

def euclidian_distance(a, b):
    return np.linalg.norm(a - b)


def tranform(data):
    model = ConvolutionalNN(data.shape[1], 10)
    model.load('model.h5')
    data = (data- np.mean(data)) / np.std(data)
    converted = model.predict_proba(data)
    return converted


def song_recommendation(song, method='cosine'):
    dataFrame = pd.DataFrame()
    for genre in genres:
        temp = pd.read_csv(f'./Data/{genre}.csv')
        dataFrame = pd.concat([dataFrame, temp], axis=0)


    ## get index of selected song
    selected_song_index = list(dataFrame['file_name'] == song).index(True)


    data = dataFrame.drop(['file_name', 'label'], axis=1)
    data = data.to_numpy()
    similarities = []

    if method == 'cosine':
        dist_func = cosine_similarity
    elif method == 'euclidian':
        dist_func = euclidian_distance
    elif method == 'neural':
        data = tranform(data)
        print(data.shape)
        dist_func = cosine_similarity
    else:
        raise Exception("Método no válido")

    selected_song = data[selected_song_index].reshape(-1, 1)

    for i in range(len(data)):
        temp = data[i].reshape(-1, 1)
        similarities.append(dist_func(temp, selected_song))
    similarities = np.array(similarities)
    if method == 'cosine' or method == 'neural':
        similarities = similarities.argsort()[-6:][::-1]
        similarities = similarities[1:]
    elif method == 'euclidian':
        similarities = similarities.argsort()[:6]
        similarities = similarities[1:]
    print("Caniones recomendadas:")
    print(similarities)
    for i in similarities:
        song = dataFrame.iloc[i]['file_name']
        genre = dataFrame.iloc[i]['label']
        print(f'{song}')
        ipd.display(ipd.Audio(f'./Data/genres_original/{genre}/{song}'))

def song_selection():
    print("0. Salir")
    for i, genre in enumerate(genres):
        print(f"{i + 1}. {genre}")
    genre = int(input("Ingrese el género de la canción> "))
    if genre == 0:
        return
    if genre < 0 or genre > len(genres):
        print("Opción inválida")
        return
    genre = genres[genre - 1]
    print(f"Género seleccionado: {genre}")
    song = int(input("Ingrese el numero de la canción> "))
    if song < 0 or song > 99:
        print("Opción inválida")
        return
    song = f'{song:05d}'
    print(f'Cancion seleccionada: {genre}.{song}.wav')
    ipd.display(ipd.Audio(f'./Data/genres_original/{genre}/{genre}.{song}.wav'))
    return f'{genre}.{song}.wav'


def recommend_ui():
    print("0. Salir")
    print("1. Por similitud de canciones")
    print("2. Por cercania de canciones")
    print("3. Por similitud de canciones (Red Neuronal)")
    option = input("Ingrese una opción> ")
    if option == '1':
        song_recommendation(song_selection(), method='cosine')
    elif option == '2':
        song_recommendation(song_selection(), method='euclidian')
    elif option == '3':
        song_recommendation(song_selection(), method='neural')
    elif option == '0':
        return
    else:
        print("Opción inválida")


def main():
    recommend_ui()


if __name__ == '__main__':
    main()

0. Salir
1. Por similitud de canciones
2. Por cercania de canciones
3. Por similitud de canciones (Red Neuronal)
